### Introduction

The goal of this notebook is to introduce TS-tools, a useful open-source software package that facilitate the automated generation of reaction profiles, starting from a simple reaction SMILES input (see figure below for a schematic overview of the algorithm; more information about the technical detail can be found in the associated paper -- https://onlinelibrary.wiley.com/doi/10.1002/jcc.27374). Compared to autodE, TS-tools is particularly suited for transition states involving 3 or more reactants, which are relevant for example in solvent- and autocatalysis, as well as enzymatic reactivity.

<div style="text-align: center;">
<img src="https://onlinelibrary.wiley.com/cms/asset/773da5e4-fbc2-47d4-a3f3-4ce4cbe7dcbb/jcc27374-fig-0002-m.jpg" alt="alt text" width="600"/>
</div>

### TS-tools

Please start by installing -- and activating -- the conda environment present in the repository (conda env create -f environment.yml)

Next, we need to install TS-tools. This can be done by accessing the package through the command line (cd TS-tools-master), followed by a local installation (pip install .)

In [ ]:
import os

home_dir = os.getcwd()

# Change to the directory containing the TS-tools package
os.chdir('TS-tools-master')

# Run the pip install command
!pip install .

os.chdir(home_dir)

A detailed explanation of how the TS-tools code can be executed can be found in the readme.md file present in the repository. To execute its full workflow, Gaussian16 needs to be available. Since Gaussian16 is not installed on your local computers, we will focus here exclusively on the generation of preliminary TS guesses from (xTB-generated) reactive paths.

### Generate TS guesses

The reaction SMILES we will try to generate TS guesses for can be found in 'TS-tools-master/data/test_aldol.txt'. They are schematically represented in the figure below.

<div style="text-align: center;">
<img src="data/aldol.png" width="500" >
</div>

In [ ]:
from tstools.ts_optimizer import TSOptimizer
import multiprocessing
import concurrent.futures
from tstools.utils import remove_files_in_directory, get_reaction_list

reaction_list = get_reaction_list('TS-tools-master/data/test_aldol.txt')

In [ ]:
# define some auxiliary functions to interact with the autodE code; these are adapted from the run_scripts available in the 'TS-tools-master' folder.
def obtain_individual_guesses(ts_optimizer):
    """
    Optimize an individual transition state.

    Parameters:
    - ts_optimizer: Instance of TSOptimizer.

    Returns:
    - int or None: Reaction ID if a transition state is found, None otherwise.
    """
    reactive_complex_factor_value = 1.8
    for _ in range(3):
        try:
            ts_optimizer.set_ts_guess_list(reactive_complex_factor_value)
            remove_files_in_directory(os.getcwd())
            if len(ts_optimizer.ts_guess_list) is not None:
                return ts_optimizer.rxn_id
        except Exception as e:
            print(e)
            continue

def obtain_ts_guesses(target_dir, reaction_list, xtb_external_path, solvent,
                             reactive_complex_factor_list_intermolecular,
                             reactive_complex_factor_list_intramolecular, freq_cut_off):
    """
    Obtain transition states for a list of reactions.

    Parameters:
    - target_dir (str): Target directory.
    - reaction_list (list): List of reactions.
    - xtb_external_path (str): Path to the XTB external script.
    - solvent (str): Solvent information.
    - reactive_complex_factor_list_intermolecular (list): List of reactive complex factors for intermolecular reactions.
    - reactive_complex_factor_list_intramolecular (list): List of reactive complex factors for intramolecular reactions.
    - freq_cut_off (int): Frequency cutoff.

    Returns:
    - list: List of successful reactions.
    """
    home_dir = os.getcwd()
    os.chdir(target_dir)
    ts_optimizer_list = []

    for rxn_idx, rxn_smiles in reaction_list:
        ts_optimizer_list.append(TSOptimizer(rxn_idx, rxn_smiles, xtb_external_path,
                                             solvent, None, reactive_complex_factor_list_intermolecular,
                                             reactive_complex_factor_list_intramolecular, freq_cut_off))

    print(f'{len(ts_optimizer_list)} reactions to process...')

    num_processes = multiprocessing.cpu_count()

    with concurrent.futures.ProcessPoolExecutor(max_workers=int(num_processes/2)) as executor:
        # Map the function to each object in parallel
        results = list(executor.map(obtain_individual_guesses, ts_optimizer_list))

    successful_reactions = [r for r in results if r is not None]

    print(f'Guesses have been generated for {len(successful_reactions)} out of 4 reactions!')

In [ ]:
os.mkdir('work_dir/ts-tools')
target_dir = 'work_dir/ts-tools' 

successful_reactions = obtain_ts_guesses(target_dir, reaction_list, 
    xtb_external_path=None, solvent=None, reactive_complex_factor_list_intermolecular=None,
    reactive_complex_factor_list_intramolecular=None, freq_cut_off=150)

You can now inspect the preliminary TS guesses extracted from the reactive paths for every reaction in the 'work_dir/ts-tools'. Note that when Gaussian16 is available, a TS search with the Berny algorithm is started from each of the guesses iteratively (together with an IRC search), until a validated TS has been found. At xTB level of theory, final/validated TSs can typically be obtained on a minute scale, DFT level transition states can typically be obtained in a matter of several hours (depending on the basis set and functional selected).